# BERT

In [1]:
!pip install sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 85 kB 2.3 MB/s 
     |████████████████████████████████| 5.8 MB 49.2 MB/s 
     |████████████████████████████████| 1.3 MB 46.1 MB/s 
     |████████████████████████████████| 182 kB 44.2 MB/s 
     |████████████████████████████████| 7.6 MB 19.1 MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=72a1f3cfb7d2d8ee8eca6668c677127972b7e491f43fdde05450be614fb1f3e6
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [2]:
import numpy as np
import pandas as pd
import torch
import transformers as ppb # pytorch transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split

from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from sklearn. preprocessing import StandardScaler

In [3]:
def cosine(u, v):
    return np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))

In [4]:
df =pd.read_csv("https://raw.githubusercontent.com/KrzysztofSzerenos/DMML2022_-Hublot/main/data/training_data.csv",)

In [5]:
sentences = df['sentence'].tolist()

In [6]:
from sentence_transformers import SentenceTransformer

In [7]:
sbert2_model =  SentenceTransformer("dangvantuan/sentence-camembert-large")

Downloading:   0%|          | 0.00/1.18k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.42k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/683 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.35G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/27.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/809k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/298 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/400 [00:00<?, ?B/s]

In [8]:
query = "I love Data Mining !"
query_vec = sbert2_model.encode([query])[0]

In [9]:
for sent in sentences:
  sim = cosine(query_vec, sbert2_model.encode([sent])[0])
  print("Sentence = ", sent, "; similarity = ", sim)

Sentence =  Les coûts kilométriques réels peuvent diverger sensiblement des valeurs moyennes en fonction du moyen de transport utilisé, du taux d'occupation ou du taux de remplissage, de l'infrastructure utilisée, de la topographie des lignes, du flux de trafic, etc. ; similarity =  0.103812486
Sentence =  Le bleu, c'est ma couleur préférée mais je n'aime pas le vert! ; similarity =  0.13428228
Sentence =  Le test de niveau en français est sur le site Internet de l'école. ; similarity =  0.03351088
Sentence =  Est-ce que ton mari est aussi de Boston? ; similarity =  0.037612684
Sentence =  Dans les écoles de commerce, dans les couloirs de places financières, il arrive aujourd'hui de croiser de jeunes adultes de 20 ou 25 ans qui prévoient d'ouvrir une maison d'hôtes "dans une quinzaine d'années". ; similarity =  0.043164864
Sentence =  voilà une autre histoire que j'ai beaucoup aimée. ; similarity =  0.21351132
Sentence =  Les médecins disent souvent qu'on doit boire un verre de vin rou

KeyboardInterrupt: ignored

In [ ]:
df_bert =pd.read_csv("https://raw.githubusercontent.com/KrzysztofSzerenos/DMML2022_-Hublot/main/data/bert2.csv",)
df_bert.shape

In [ ]:
df['bert'] = df_bert['bert']
df.tail(10)

In [ ]:
df['word_count'] = df['sentence'].apply(lambda n: len(n.split()))
df['characters_count'] = df['sentence'].apply(lambda x: sum(len(word) for word in str(x).split(" ")))
df.head()

# Encoder

In [ ]:
oe = OrdinalEncoder()
oe.set_params(categories=[['A1', 'A2', 'B1','B2','C1','C2']])
oe_customer_class = oe.fit_transform(df[['difficulty']])
oe.categories_

In [ ]:
oe_customer_class = pd.DataFrame(oe_customer_class, columns=['oe_difficulty']).astype('int')

In [ ]:
df = pd.concat([df, oe_customer_class], axis=1) 
df

# vectorization

Linear regression model

In [ ]:
X = df[['bert','word_count','characters_count']]
y = df['oe_difficulty']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.1, random_state=0)
np.unique(y, return_counts=True)

In [ ]:
df.reset_index(drop=True, inplace=True)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()  # vectorization
vectorizer.fit(X,y)
X_train_vec = vectorizer.transform(X_train)
X_test_vec = vectorizer.transform(X_test)

In [ ]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
classifier = LogisticRegressionCV()
classifier.fit(X_train_vec, y_train)
log_reg_pred = classifier.predict(X_kag_set)

In [ ]:
submission_dataframe = pd.DataFrame(log_reg_pred, columns=['Difficulty'])
print(submission_dataframe)
submission_dataframe.to_csv('HublotKAGGLECV.csv')